In [1]:
from statsmodels.tsa.seasonal import seasonal_decompose
import re
import pickle
import numpy as np
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import json
from scipy.stats import linregress
from sklearn.cluster import KMeans

In [2]:
dates_times_path = 'dates_times.pickle'
density_lines_path = 'trend_lines.pickle'
json_path = 'json_geo.json'
animation_path = 'animation.pickle'

In [35]:
with open(density_lines_path, 'rb') as handle:
    b = pickle.load(handle)
df_dict = b
with open(dates_times_path, 'rb') as handle:
    b = pickle.load(handle)
dates_times = b
with open(animation_path, 'rb') as handle:
    b = pickle.load(handle)
df_geo = b
with open(json_path) as data_file:
    new_jfile = json.load(data_file)
    
dict_keys = []
dict_keys = df_dict.keys()

125


In [36]:
key = '40.029717525412, -105.26275634766'
key2 = '40.004475801858, -105.25177001953'
x_space = len(dates_times)/3
x = np.linspace(0,x_space,int(x_space))
slopes = []
keys = []
for key in df_dict.keys():
    keys.append(key)
    y = df_dict[key]['0000'].astype(float) 
    slopes.append([linregress(x,y).slope,0])
kmeans = KMeans(n_clusters=25, random_state=0).fit(slopes)
clusters = {}
for key, label in zip(keys, kmeans.labels_):
    clusters[key] = label


In [37]:
colorsIdx = {}
discrete_colors = []
discrete_colors.extend(px.colors.qualitative.Alphabet)
#discrete_colors.extend(px.colors.qualitative.Plotly)
#discrete_colors.extend(px.colors.qualitative.)
i = 0
for color in discrete_colors:
    colorsIdx[i] = color
    i = i + 1

In [39]:
fig = make_subplots(rows=1, cols=1, subplot_titles = ("0000 UTC", "0800 UTC", "1600 UTC"))
i = 0
added_clusters = []
for key in df_dict.keys():
    if key in dict_keys:
        df = df_dict[key]
        df['0000'] = df['0000'].astype(float)
        df['0800'] = df['0800'].astype(float)
        df['1600'] = df['1600'].astype(float)
        dates = df['dates']


        c_0 = df['0000'].to_list()
        c_8 = df['0800'].to_list()
        c_16 = df['1600'].to_list()

        j = 0
        if clusters[key] not in added_clusters:
            added_clusters.append(clusters[key])
            for c in [c_0]:
                result_mul = seasonal_decompose(c,
                                     period=7,
                                     model = 'multiplicative',
                                     extrapolate_trend='freq')

                s  = result_mul.seasonal
                t  = result_mul.trend
                t  = np.array(result_mul.trend)
                t = t - t[0]
                fig.add_trace(go.Scatter(x = dates, y = t, mode = 'lines',  customdata = df_geo['ids'],line = dict(color = 'green'),opacity = 0.75, showlegend = False),row = 1, col = 1)
                i = i + 1
                j = j + 1
        else:
            j = 0
            for c in [c_0, c_8, c_16]:
                result_mul = seasonal_decompose(c,
                                     period=7,
                                     model = 'multiplicative',
                                     extrapolate_trend='freq')

                s  = result_mul.seasonal
                t  = result_mul.trend
                t  = np.array(result_mul.trend)
                t = t - t[0]
                fig.add_trace(go.Scatter(x = dates, y = t, mode = 'lines',  customdata = df_geo['ids'], line = dict(color = 'white'),opacity = 0.1, showlegend = False),row = 1, col = 1)
                i = i + 1
                j = j + 1


fig.update_layout(template = 'plotly_dark')
fig.update_xaxes(title_text="Date", row=1, col=1)
fig.update_xaxes(title_text="Date", row=2, col=1)
fig.update_xaxes(title_text="Date", row=3, col=1)
fig.update_yaxes(title_text="Crisis Density", row=1, col=1)
fig.update_yaxes(title_text="Crisis Density", row=2, col=1)
fig.update_yaxes(title_text="Crisis Density", row=3, col=1)
fig.show()